In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import datetime as dt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [4]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'e'
project = 'agrupaciones'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')

In [5]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')
compras.head()

,cliente,monto,cuotas,fecha,dias_mora
2,Ana,2000,6,2022-01-01,25
3,Ana,3000,12,2022-03-01,0
6,Ana,5000,24,2022-03-01,8
7,Ana,300,2,2023-02-01,7
0,Juan,1000,12,2022-01-01,0


# AIMS

1. Antiguedad de la cuenta: Dias desde la creacion de la cuenta hasta la fecha.
2. Ciclo de vida: Dias desde primer compra a ultima compra.
3. Tiempo promedio y mediana entre compras: Dias promedio o mediana entre compras
4. Costo de vida: Costo de vida

## 1. Antiguedad de la cuenta

In [5]:
compras = compras.sort_values(by=['fecha'], ascending=False)
compras = compras.groupby('cliente').head(1).reset_index()
compras['antiguedad'] = (dt.datetime.now() - compras['fecha']).dt.days
compras

,index,cliente,monto,cuotas,fecha,dias_mora,antiguedad
0,7,Ana,300,2,2023-02-01,7,92
1,1,Juan,5000,24,2022-02-01,0,457
2,5,Pedro,2000,12,2022-02-01,0,457


## 2. Ciclo de vida

In [6]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

In [7]:
# crear una nueva columna con la fecha de inicio del ciclo de vida de cada cliente
fecha_inicio = compras.groupby('cliente')['fecha'].min()
fecha_inicio

cliente
Ana     2022-01-01
Juan    2022-01-01
Pedro   2022-01-01
Name: fecha, dtype: datetime64[ns]

In [8]:
# agregar el inicio del ciclo de vida
compras['inicio_ciclo_vida'] = compras['cliente'].map(fecha_inicio)
compras

,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida
2,Ana,2000,6,2022-01-01,25,2022-01-01
3,Ana,3000,12,2022-03-01,0,2022-01-01
6,Ana,5000,24,2022-03-01,8,2022-01-01
7,Ana,300,2,2023-02-01,7,2022-01-01
0,Juan,1000,12,2022-01-01,0,2022-01-01
1,Juan,5000,24,2022-02-01,0,2022-01-01
4,Pedro,1500,8,2022-01-01,11,2022-01-01
5,Pedro,2000,12,2022-02-01,0,2022-01-01


In [9]:
# crear una nueva columna con la fecha de fin del ciclo de vida de cada cliente
fecha_fin = compras.groupby('cliente')['fecha'].max()
fecha_fin

cliente
Ana     2023-02-01
Juan    2022-02-01
Pedro   2022-02-01
Name: fecha, dtype: datetime64[ns]

In [10]:
compras['fin_ciclo_vida'] = compras['cliente'].map(fecha_fin)
compras

,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida,fin_ciclo_vida
2,Ana,2000,6,2022-01-01,25,2022-01-01,2023-02-01
3,Ana,3000,12,2022-03-01,0,2022-01-01,2023-02-01
6,Ana,5000,24,2022-03-01,8,2022-01-01,2023-02-01
7,Ana,300,2,2023-02-01,7,2022-01-01,2023-02-01
0,Juan,1000,12,2022-01-01,0,2022-01-01,2022-02-01
1,Juan,5000,24,2022-02-01,0,2022-01-01,2022-02-01
4,Pedro,1500,8,2022-01-01,11,2022-01-01,2022-02-01
5,Pedro,2000,12,2022-02-01,0,2022-01-01,2022-02-01


In [11]:
# calcular la duración del ciclo de vida de cada cliente
compras['ciclo_vida'] = (pd.to_datetime(compras['fin_ciclo_vida']) - pd.to_datetime(compras['inicio_ciclo_vida'])).dt.days
compras

,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida,fin_ciclo_vida,ciclo_vida
2,Ana,2000,6,2022-01-01,25,2022-01-01,2023-02-01,396
3,Ana,3000,12,2022-03-01,0,2022-01-01,2023-02-01,396
6,Ana,5000,24,2022-03-01,8,2022-01-01,2023-02-01,396
7,Ana,300,2,2023-02-01,7,2022-01-01,2023-02-01,396
0,Juan,1000,12,2022-01-01,0,2022-01-01,2022-02-01,31
1,Juan,5000,24,2022-02-01,0,2022-01-01,2022-02-01,31
4,Pedro,1500,8,2022-01-01,11,2022-01-01,2022-02-01,31
5,Pedro,2000,12,2022-02-01,0,2022-01-01,2022-02-01,31


In [12]:
compras = compras.groupby('cliente').head(1).reset_index()
compras

,index,cliente,monto,cuotas,fecha,dias_mora,inicio_ciclo_vida,fin_ciclo_vida,ciclo_vida
0,2,Ana,2000,6,2022-01-01,25,2022-01-01,2023-02-01,396
1,0,Juan,1000,12,2022-01-01,0,2022-01-01,2022-02-01,31
2,4,Pedro,1500,8,2022-01-01,11,2022-01-01,2022-02-01,31


## 3. Tiempo promedio y mediana entre creditos

In [13]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

In [14]:
# agrupar los créditos por ID de cliente
grupo_clientes = compras.groupby('cliente')
grupo_clientes

In [15]:
# calcular la diferencia en días entre cada crédito y el siguiente crédito
diferencia_dias = grupo_clientes['fecha'].diff().dt.days
diferencia_dias

2      NaN
3     59.0
6      0.0
7    337.0
0      NaN
1     31.0
4      NaN
5     31.0
Name: fecha, dtype: float64

In [16]:
# calcular el promedio de la diferencia en días para cada cliente
promedio_dias = diferencia_dias.groupby(compras['cliente']).mean()
promedio_dias

cliente
Ana      132.0
Juan      31.0
Pedro     31.0
Name: fecha, dtype: float64

In [17]:
# calcular la mediana de la diferencia en días para cada cliente
promedio_dias = diferencia_dias.groupby(compras['cliente']).median()
promedio_dias

cliente
Ana      59.0
Juan     31.0
Pedro    31.0
Name: fecha, dtype: float64

## 4. Costo de vida

In [18]:
compras = pd.read_parquet(processed / 'compras.parquet.gzip')

In [19]:
# agrupar las transacciones del cliente por ID
grupo_clientes = compras.groupby('cliente')
grupo_clientes

In [20]:
# calcular la fecha de la última transacción y el tiempo transcurrido desde la primera
fecha_ultima_trans = grupo_clientes['fecha'].max()
tiempo_transcurrido = (fecha_ultima_trans - grupo_clientes['fecha'].min()).dt.days

In [21]:
# calcular el número de transacciones del cliente y el valor promedio de la transacción
num_transacciones = grupo_clientes['monto'].count()
valor_promedio = grupo_clientes['monto'].mean()

In [22]:
# definir la tasa de descuento
tasa_descuento = 0.1

In [23]:
# calcular el CLV utilizando la fórmula de VPN
clv = (valor_promedio * num_transacciones * (1 - np.exp(-tasa_descuento * tiempo_transcurrido))) / tasa_descuento

# imprimir el resultado
print(clv)

cliente
Ana      103000.000000
Juan      57297.047856
Pedro     33423.277916
dtype: float64


In [24]:
print('ok_')

ok_
